In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

import concurrent.futures
from selenium import webdriver

from dotenv import load_dotenv
import os
import queue
import threading

import betfairlightweight
from betfairlightweight import filters
from betfairlightweight import APIClient

#### Oddschecker data
First extract the urls containing each of the odds tables from the oddschecker politics sitemap at https://www.oddschecker.com/sport/politics/sitemap.xml. However this does appear to be missing a bunch of markets for some reason, so it's simpler to manually update the list of markets

In [2]:
# Extract oddschecker politics market urls from sitemap
chrome_options = Options()
chrome_options.add_argument("--headless") 
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

service = Service("/opt/homebrew/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=chrome_options)

# URL of the sitemap
sitemap_url = "https://www.oddschecker.com/sport/politics/sitemap.xml"

try:
    # Load the sitemap page
    driver.get(sitemap_url)

    time.sleep(5)
    
    xml_content = driver.page_source
    soup = BeautifulSoup(xml_content, 'xml')

    url_tags = soup.find_all('loc')
    urls = [url_tag.text for url_tag in url_tags]

    print(f"Found {len(urls)} URLs.")
    print(urls)

finally:
    # Close the browser
    driver.quit()

Found 0 URLs.
[]


Selenium is then used to load the pages and grab the page source. It's currently set to run five pages concurrently (using different headers) to help speed things up. The odds table is then extracted using beautifulsoup, putting the bet name and odds for each bookmaker into a dataframe. The data for the odds of each bookmaker seems to have a somewhat random class name in the html, but most contain "bs" or "o", so this is what's searched for in the table info.

In [3]:
# Function to extract odds data from a given URL
def extract_odds(url, user_agent):
    chrome_options = Options()
    chrome_options.add_argument(f'user-agent={user_agent}')
    chrome_options.add_argument("--headless")

    service = Service("/opt/homebrew/bin/chromedriver")
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        # Navigate to the specific oddschecker page
        driver.get(url)

        # Wait for the page to load
        time.sleep(6) 

        # Get the page source
        page_source = driver.page_source

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        # Odds table has id "t1"
        odds_table = soup.find('tbody', id='t1')

        if not odds_table:
            print(f"No odds table found for URL: {url}")
            return None  # Skip this URL if the table isn't found

        # Extract each row and the data within
        odds_data = []
        bookmakers_set = set()

        for row in odds_table.find_all('tr'):
            bet_name = row.find('a', class_='popup').text.strip() 
            odds_dict = {'Bet': bet_name}
            
            # Find all td elements with odds information
            for td in row.find_all('td', class_=lambda x: x and ('o' in x.split() or 'bs' in x.split())): 
                bookmaker = td.get('data-bk')  # Extract the bookmaker name
                decimal_odds = td.get('data-odig')  # Extract the decimal odds value
                if bookmaker and decimal_odds:  # Only add if both are present
                    odds_dict[bookmaker] = float(decimal_odds)  # Convert odds to float
                    bookmakers_set.add(bookmaker)
            
            odds_data.append(odds_dict)

        # Create a DataFrame with all bookmakers as columns
        df = pd.DataFrame(odds_data).set_index('Bet')

        # Ensure all bookmakers are columns, even if some are missing in certain rows
        df = df.reindex(columns=sorted(bookmakers_set))

        # Add the URL as a column in the DataFrame
        df['URL'] = url

        return df
    finally:
        # Close the browser
        driver.quit()

In [4]:
# The oddschecker sitemap seems to be missing some pages, so manually updated urls
urls = [
    "https://www.oddschecker.com/politics/british-politics/next-labour-leader",
    "https://www.oddschecker.com/politics/british-politics/next-conservative-leader",
    "https://www.oddschecker.com/politics/australian-politics/state-elections/queensland-state-election",
    "https://www.oddschecker.com/politics/us-politics/us-presidential-election/winner",
    "https://www.oddschecker.com/politics/us-politics/us-presidential-election/winning-party",
    "https://www.oddschecker.com/politics/us-politics/us-presidential-election/party-of-popular-vote-winner",
    "https://www.oddschecker.com/politics/us-politics/us-presidential-election/gender-of-election-winner",
    "https://www.oddschecker.com/politics/us-politics/us-presidential-election/election-winner-to-lose-popular-vote",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/mississippi",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/arizona",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/massachusetts",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/oklahoma",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/pennsylvania",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/oregon",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/minnesota",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/hawaii",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/alabama",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/texas",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/rhode-island",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/florida",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/delaware",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/connecticut",
    "https://www.oddschecker.com/politics/us-politics/us-state-betting/colorado"
]

# List of user agents to rotate
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15'
]

# List to store DataFrames
dataframes_list_oc = []

# Use ThreadPoolExecutor to process URLs in parallel in batches of 5
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = []
    for i, url in enumerate(urls):
        user_agent = user_agents[i % len(user_agents)]  # Rotate user agents
        futures.append(executor.submit(extract_odds, url, user_agent))

        # Wait for each batch of 5 to complete before starting the next batch
        if (i + 1) % 5 == 0 or i == len(urls) - 1:
            for future in concurrent.futures.as_completed(futures):
                df = future.result()
                if df is not None:
                    dataframes_list_oc.append(df)
            futures = []  # Clear futures list for the next batch

In [5]:
all_data_oc = pd.concat(dataframes_list_oc).reset_index()
all_data_oc.rename(columns={'index': 'Bet'}, inplace=True)

print(all_data_oc)

               Bet   AKB      B3      BF     BY      CE      DP     EE  \
0      Republicans  0.00    1.57    1.67   0.00    1.61    1.60   1.62   
1         Democrat  0.00    2.38    2.43   0.00    2.30    2.40   2.38   
2      Independent  0.00  131.00  147.00   0.00  101.00  101.00  51.00   
3     Donald Trump  1.66    1.57    1.68   1.57    1.61    1.60   1.62   
4    Kamala Harris  2.50    2.38    2.43   2.40    2.30    2.40   2.38   
..             ...   ...     ...     ...    ...     ...     ...    ...   
151    Republicans  0.00   15.00    0.00  10.00    9.00    9.00   0.00   
152      Democrats  0.00    1.01    0.00   1.01    1.01    1.01   0.00   
153    Republicans  0.00   26.00    0.00  19.00   17.00   26.00   0.00   
154      Democrats  0.00    1.01    0.00   1.01    1.01    1.01   0.00   
155    Republicans  0.00   26.00    0.00  19.00   15.00   21.00   0.00   

         FB      FR  ...    S6     SI      SK     SX      UN     VC      VT  \
0      1.62    1.62  ...  0.00  

#### Betfair data
Next to grab the data from the Betfair Exchange API for political markets. 

In [110]:
# Load login credentials
load_dotenv()

bf_usr = os.getenv("BF_LOGIN")
bf_pass = os.getenv("BF_PASS")
bf_api = os.getenv("BF_API_KEY")
#bf_session = os.getenv("BF_SESSION")
bf_certs_path = '../certs/'

In [111]:
# Login to betfair client
client = APIClient(bf_usr, bf_pass, app_key=bf_api, certs=bf_certs_path)
client.login()

<LoginResource>

The market ID's are extracted given the event ID for politcal bets, followed by getting the bid/ask price and size for each market

In [112]:
# Fetch Market Catalogues and create mappings for Selection and Market Names
market_filter = betfairlightweight.filters.market_filter(
    event_type_ids=['2378961'],  # Politics event type
)

# Get market catalogues, including runners
market_catalogues = client.betting.list_market_catalogue(
    filter=market_filter,
    max_results=100,  # Adjust this as needed
    market_projection=['RUNNER_DESCRIPTION']  # Include runner descriptions to get selection names
)

# Extract market IDs and create mappings
market_ids = [market.market_id for market in market_catalogues]

selection_mapping = {}
market_name_mapping = {}

for market in market_catalogues:
    market_name_mapping[market.market_id] = market.market_name  # Map market_id to market_name
    for runner in market.runners:
        selection_mapping[runner.selection_id] = runner.runner_name  # Map selection_id to selection_name

In [113]:
# Function to process runner books and include selection and market names
def process_runner_books(runner_books, selection_mapping, market_name, market_id):
    best_back_prices = [
        runner_book.ex.available_to_back[0]['price'] if runner_book.ex.available_to_back else 1.01
        for runner_book in runner_books
    ]
    best_back_sizes = [
        runner_book.ex.available_to_back[0]['size'] if runner_book.ex.available_to_back else 1.01
        for runner_book in runner_books
    ]

    best_lay_prices = [
        runner_book.ex.available_to_lay[0]['price'] if runner_book.ex.available_to_lay else 1000.0
        for runner_book in runner_books
    ]
    best_lay_sizes = [
        runner_book.ex.available_to_lay[0]['size'] if runner_book.ex.available_to_lay else 1.01
        for runner_book in runner_books
    ]

    selection_ids = [runner_book.selection_id for runner_book in runner_books]
    selection_names = [selection_mapping.get(runner_book.selection_id, "Unknown") for runner_book in runner_books]
    last_prices_traded = [runner_book.last_price_traded for runner_book in runner_books]
    total_matched = [runner_book.total_matched for runner_book in runner_books]
    statuses = [runner_book.status for runner_book in runner_books]
    scratching_datetimes = [runner_book.removal_date for runner_book in runner_books]
    adjustment_factors = [runner_book.adjustment_factor for runner_book in runner_books]

    market_id = str(market_id)

    df = pd.DataFrame({
        'Market ID': market_id,
        'Market Name': market_name,
        'Selection ID': selection_ids,
        'Selection Name': selection_names,
        'Best Back Price': best_back_prices,
        'Best Back Size': best_back_sizes,
        'Best Lay Price': best_lay_prices,
        'Best Lay Size': best_lay_sizes,
        'Last Price Traded': last_prices_traded,
        'Total Matched': total_matched,
        'Status': statuses,
        'Removal Date': scratching_datetimes,
        'Adjustment Factor': adjustment_factors
    })
    return df

In [114]:
# Create a price filter for market data
price_filter = betfairlightweight.filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# List to store DataFrames
dataframes_list_bf = []

# Loop through each market ID and fetch market book data
for market_id in market_ids:
    # Request market book for each market ID
    market_books = client.betting.list_market_book(
        market_ids=[market_id],
        price_projection=price_filter
    )
    
    # Ensure that market books were returned
    if market_books:
        # Process the first market book (only one is requested)
        market_book = market_books[0]
        
        # Get market name using the market_id
        market_name = market_name_mapping[market_id]
        
        # Process runner books and store in DataFrame, including selection names and market names
        runners_df = process_runner_books(market_book.runners, selection_mapping, market_name, market_id)
        
        # Append the DataFrame to the list
        dataframes_list_bf.append(runners_df)

# Optionally, you can concatenate all dataframes into a single dataframe
all_data_df = pd.concat(dataframes_list_bf, ignore_index=True)

# Display or process the combined DataFrame as needed
print(all_data_df)

       Market ID         Market Name  Selection ID      Selection Name  \
0    1.170273835  Next Labour Leader      11149003       Wes Streeting   
1    1.170273835  Next Labour Leader       5859542       Rachel Reeves   
2    1.170273835  Next Labour Leader       2601290        Andy Burnham   
3    1.170273835  Next Labour Leader      28275586  Bridget Phillipson   
4    1.170273835  Next Labour Leader       1288344       Yvette Cooper   
..           ...                 ...           ...                 ...   
579  1.229997508              Kansas       1171580         Republicans   
580  1.230123858               Texas       1171581           Democrats   
581  1.230123858               Texas       1171580         Republicans   
582  1.229997507                Iowa       1171581           Democrats   
583  1.229997507                Iowa       1171580         Republicans   

     Best Back Price  Best Back Size  Best Lay Price  Best Lay Size  \
0               5.10           14.99    

/var/folders/ss/kjxldrfs36qgygzmg25sqy100000gn/T/ipykernel_14957/2125063919.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_data_df = pd.concat(dataframes_list_bf, ignore_index=True)


### Bet matching
An attempt to automate matching the market names of betfair to oddschecker using cosine similarity. It has some success but as it's unlikely that markets will be added or removed frequently, it'll probably be easier to manually match the markets

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Extract market names from Betfair data
betfair_market_names = list(set(all_data_df['Market Name'].tolist()))

# Combine all market names for vectorization
all_market_names = betfair_market_names + urls

# Vectorize the market names using TF-IDF
vectorizer = TfidfVectorizer().fit_transform(all_market_names)
vectors = vectorizer.toarray()

# Calculate cosine similarity between Betfair and Oddschecker markets
cosine_sim_matrix = cosine_similarity(vectors[:len(betfair_market_names)], vectors[len(betfair_market_names):])

# Find the best matches for each Betfair market
matches = []
for i, betfair_name in enumerate(betfair_market_names):
    similarity_scores = cosine_sim_matrix[i]
    best_match_idx = np.argmax(similarity_scores)
    best_match_score = similarity_scores[best_match_idx]
    best_match_name = urls[best_match_idx]
    matches.append({
        'Betfair Market Name': betfair_name,
        'Oddschecker Market Name': best_match_name,
        'Similarity Score': best_match_score
    })

# Convert matches to DataFrame for easier review
matches_df = pd.DataFrame(matches)

# Filter out rows with a similarity score of 0
matches_df_filtered = matches_df[matches_df['Similarity Score'] > 0]

# Sort the DataFrame by similarity score in descending order
matches_df_filtered = matches_df_filtered.sort_values(by='Similarity Score', ascending=False)

# Display the sorted DataFrame
print(matches_df_filtered)

The mappings are loaded from data/markets.csv as opposed to the cosine similarity matching above. A fuzzy match is then carried out between the 'Selection Name' from the betfair data and 'Bet' from the oddschecker data as the name of the Bets may not be identical.

In [ ]:
from rapidfuzz import fuzz
from rapidfuzz import process

# Load the mapping CSV file and ensure Market ID is a string for matching consistency
mapping_df = pd.read_csv("../data/markets.csv")
mapping_df['Market ID'] = mapping_df['Market ID'].astype(str)

# Load the first DataFrame (Betfair data) and ensure Market ID is a string
betfair_df = all_data_df
betfair_df['Market ID'] = betfair_df['Market ID'].astype(str)

# Load the second DataFrame (Oddschecker data)
oddschecker_df = all_data_oc

# Normalize the 'Selection Name' and 'Bet' columns for better matching
betfair_df['Selection Name Normalized'] = betfair_df['Selection Name'].str.lower().str.strip()
oddschecker_df['Bet Normalized'] = oddschecker_df['Bet'].str.lower().str.strip()

# Initialize a list to collect all results
all_results = []

# Iterate over each market in the mapping DataFrame
for index, row in mapping_df.iterrows():
    market_id = row['Market ID']
    url = row['URL']
    
    # Skip markets without a URL
    if pd.isna(url) or url.strip() == "":
        continue
    
    # Filter Betfair DataFrame to only include rows with the current Market ID
    betfair_filtered_df = betfair_df[betfair_df['Market ID'] == market_id]
    
    # Filter Oddschecker DataFrame to only include rows with the current URL
    oddschecker_filtered_df = oddschecker_df[oddschecker_df['URL'] == url]
    
    # Skip if there are no corresponding entries in either filtered DataFrame
    if betfair_filtered_df.empty or oddschecker_filtered_df.empty:
        continue
    
    # Create a function to find the best match for each 'Selection Name'
    def match_bets(selection_name, bet_choices):
        match, score, _ = process.extractOne(selection_name, bet_choices, scorer=fuzz.token_sort_ratio)
        return match, score

    # Apply the matching function to each 'Selection Name' in the filtered Betfair DataFrame
    bet_choices = oddschecker_filtered_df['Bet Normalized'].tolist()
    matches = betfair_filtered_df['Selection Name Normalized'].apply(lambda x: match_bets(x, bet_choices))

    # Add match results to betfair_filtered_df
    betfair_filtered_df['Best Match Bet'] = matches.apply(lambda x: x[0])
    betfair_filtered_df['Similarity Score'] = matches.apply(lambda x: x[1])

    # Filter matches based on a similarity threshold (e.g., 80)
    threshold = 80
    filtered_matches_df = betfair_filtered_df[betfair_filtered_df['Similarity Score'] >= threshold]
    
    # Merge with the filtered Oddschecker DataFrame on 'URL' and 'Best Match Bet'
    final_filtered_df = pd.merge(filtered_matches_df, oddschecker_filtered_df, left_on=['Best Match Bet'], right_on=['Bet'], how='left')
    
    # Drop temporary columns and finalize DataFrame
    final_filtered_df = final_filtered_df.drop(columns=['Selection Name Normalized', 'Bet Normalized', 'Best Match Bet', 'Similarity Score'])
    
    # Append the results to the all_results list
    all_results.append(final_filtered_df)

# Concatenate all the DataFrames in the list to create a single DataFrame
final_df = pd.concat(all_results, ignore_index=True)

# Display or save the final DataFrame as needed
print(final_df)

### Joining tables

In [16]:
# Load the mapping CSV file and ensure Market ID is a string for matching consistency
mapping_df = pd.read_csv("../data/markets.csv")
mapping_df['Market ID'] = mapping_df['Market ID'].astype(str)

# Load the first DataFrame (Betfair data) and ensure Market ID is a string
betfair_df = all_data_df
betfair_df['Market ID'] = betfair_df['Market ID'].astype(str)

# Load the second DataFrame (Oddschecker data)
oddschecker_df = all_data_oc

# Normalize the 'Selection Name' and 'Bet' columns for better matching
betfair_df['Selection Name Normalized'] = betfair_df['Selection Name'].str.lower().str.strip()
oddschecker_df['Bet Normalized'] = oddschecker_df['Bet'].str.lower().str.strip()

# Initialize a list to collect all results
all_results = []

# Iterate over each market in the mapping DataFrame
for index, row in mapping_df.iterrows():
    market_id = row['Market ID']
    url = row['URL']
    
    # Skip markets without a URL
    if pd.isna(url) or url.strip() == "":
        continue
    
    # Filter Betfair DataFrame to only include rows with the current Market ID
    betfair_filtered_df = betfair_df[betfair_df['Market ID'] == market_id]
    
    # Filter Oddschecker DataFrame to only include rows with the current URL
    oddschecker_filtered_df = oddschecker_df[oddschecker_df['URL'] == url]
    
    # Skip if there are no corresponding entries in either filtered DataFrame
    if betfair_filtered_df.empty or oddschecker_filtered_df.empty:
        continue
    
    # Perform direct matching of 'Selection Name Normalized' with 'Bet Normalized'
    merged_df = pd.merge(
        betfair_filtered_df,
        oddschecker_filtered_df,
        left_on='Selection Name Normalized',
        right_on='Bet Normalized',
        how='inner'
    )
    
    # Drop unnecessary columns from the merged DataFrame
    merged_df = merged_df.drop(columns=['Selection Name Normalized', 'Bet Normalized'])
    
    # Append the results to the all_results list
    all_results.append(merged_df)

# Concatenate all the DataFrames in the list to create a single DataFrame
odds_df = pd.concat(all_results, ignore_index=True)

In [17]:
def identify_arbitrage(df):
    odds_columns = ['AKB', 'B3', 'BF', 'BY', 'CE', 'DP', 'EE', 'FB', 'FR', 'G5', 'KN', 'LD', 'LS', 'MA', 'N4', 'OE', 'PP', 'QN', 'S6', 'SI', 'SK', 'SX', 'UN', 'VC', 'VT', 'WA', 'WH']
    #select_columns = ['Market Name', 'Selection Name', 'Odds to Lay Ratio', 'Best Back Price', 'Best Lay Price', 'Best Lay Size', 'Best Odds', 'Best Bookmaker', 'URL']

    # Find the best odds and corresponding bookmaker
    df['Best Odds'] = df[odds_columns].max(axis=1)
    df['Best Bookmaker'] = df.apply(lambda row: ', '.join([col for col in odds_columns if row[col] == row['Best Odds']]), axis=1)

    # Calculate the ratio of Best Odds to Best Lay Price
    df['Odds to Lay Ratio'] = df.apply(lambda row: row['Best Odds'] / row['Best Lay Price'] if row['Best Odds'] <= 500 else 0, axis=1)

    # Sort the DataFrame by the 'Odds to Lay Ratio' in descending order
    df = df.sort_values(by='Odds to Lay Ratio', ascending=False)

    # Convert necessary columns to float for calculations
    df['Best Odds'] = df['Best Odds'].astype(float)
    df['Best Lay Price'] = df['Best Lay Price'].astype(float)
    df['Best Lay Size'] = df['Best Lay Size'].astype(float)

    # Calculate Lay Liability
    df['Lay Liability'] = (df['Best Lay Size'] * (df['Best Lay Price'] - 1)).round(2)

    # Calculate Back Amount (B) for hedging
    df['Back Amount'] = (df['Best Lay Size'] * df['Best Lay Price'] / (df['Best Odds'])).round(2)

    # Calculate Profit
    df['Profit If Outcome Happens'] = (df['Back Amount'] * (df['Best Odds'] - 1)) - df['Lay Liability']
    df['Profit If Outcome Does Not Happen'] = df['Best Lay Size'] - df['Back Amount']

    # Set Profit to the minimum of the two scenarios, rounded to 2 decimal places
    df['Profit'] = df[['Profit If Outcome Happens', 'Profit If Outcome Does Not Happen']].min(axis=1).round(2)

    # Set Profit to 0 if Best Odds > 500
    df.loc[df['Best Odds'] > 500, 'Profit'] = 0

    # Filter for potential arbitrage opportunities
    arb_opportunities = df[df['Odds to Lay Ratio'] > 1]

    # Select and order the columns for the final DataFrame
    cols = ['Market Name', 'Selection Name', 'Odds to Lay Ratio', 'Best Odds', 'Best Lay Price', 'Best Lay Size', 'Best Bookmaker',
            'Lay Liability', 'Back Amount', 'Profit', 'URL']

    return arb_opportunities[cols]

arbitrage_opportunities = identify_arbitrage(odds_df)

In [18]:
from IPython.display import display
display(arbitrage_opportunities)

,Market Name,Selection Name,Odds to Lay Ratio,Best Odds,Best Lay Price,Best Lay Size,Best Bookmaker,Lay Liability,Back Amount,Profit,URL
8,Next Labour Leader,Darren Jones,1.340000,67.00,50.00,10.60,"FB, PP",519.40,7.91,2.66,https://www.oddschecker.com/politics/british-p...
58,Election Winner,Kamala Harris,1.008065,2.50,2.48,2402.36,AKB,3555.49,2383.14,19.22,https://www.oddschecker.com/politics/us-politi...
81,Will Election Winner lose Popular Vote?,Yes,1.003650,2.75,2.74,148.85,AKB,259.00,148.31,0.54,https://www.oddschecker.com/politics/us-politi...


### Add Polymarket and Predictit
#### Predictit

In [45]:
import aiohttp
import json

# PredictIt API function
async def fetch_predictit_data():
    url = "https://www.predictit.org/api/marketdata/all/"
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            data = await response.json()
    return data

In [47]:
predictit_data = await fetch_predictit_data()

predictit_df = pd.json_normalize(predictit_data['markets'])

print(predictit_df)

     id                                               name  \
0  6867  Which party will win the 2024 U.S. presidentia...   
1  7013    Will a woman be elected U.S. president in 2024?   
2  7136       Will Joe Biden resign during his first term?   
3  7419       Will Kamala Harris be the 47th US president?   
4  7456    Who will win the 2024 US presidential election?   
5  8070  Which party will win the 2024 US Senate electi...   
6  8072  Which party will win Georgia in the 2024 presi...   
7  8075  Which party will win the 2024 US Senate electi...   
8  8076  Which party will win Wisconsin in the 2024 pre...   
9  8077  What will be the Electoral College margin in t...   

                                  shortName  \
0  Which party wins the presidency in 2024?   
1          Woman president elected in 2024?   
2  Will Biden resign during his first term?   
3             Harris the 47th US president?   
4        2024 presidential election winner?   
5  Which party will win the AZ Sena

#### Polymarket
ClobClient connection:

In [31]:
from py_clob_client.client import ClobClient

# Polymarket API function
host = "https://clob.polymarket.com"
key = os.getenv("PK")
chain_id = 137  # Polygon Mainnet chain ID
# Ensure the private key is loaded correctly
if not key:
    raise ValueError("Private key not found. Please set PK in the environment variables.")
# Initialize the client with your private key
client = ClobClient(host, key=key, chain_id=chain_id)


Market data from https://gamma-api.polymarket.com

In [115]:
import pandas as pd
import requests
import ast

# Fetch data from the Gamma API
r = requests.get("https://gamma-api.polymarket.com/events?closed=false")
response = r.json()

# Function to extract overall market data
def extract_overall_market_data(market):
    return {
        'Market ID': market.get('id', 'N/A'),
        'Title': market.get('title', 'N/A'),
        'End Date': market.get('endDate', 'N/A'),
        'Overall Liquidity': float(market.get('liquidity', 0)),
        'Overall Volume': float(market.get('volume', 0)),
        'Overall Volume 24hr': float(market.get('volume24hr', 0)),
    }

# Function to extract individual bet data
def extract_bet_data(bet):
    outcome_prices_str = bet.get('outcomePrices', '["N/A", "N/A"]')
    try:
        outcome_prices = ast.literal_eval(outcome_prices_str)
        yes_price = float(outcome_prices[0]) if len(outcome_prices) > 0 else None
        no_price = float(outcome_prices[1]) if len(outcome_prices) > 1 else None
    except (ValueError, SyntaxError):
        yes_price = None
        no_price = None

    return {
        'Bet ID': bet.get('id'),
        'Bet Question': bet.get('question', 'N/A'),
        'Bet Liquidity': float(bet.get('liquidity', 0)),
        'Bet Volume': float(bet.get('volume', 0)),
        'Bet Volume 24hr': float(bet.get('volume24hr', 0)),
        'Yes Price': yes_price,
        'No Price': no_price,
        'CLOB Token ID': bet.get('clobTokenIds')
    }

# Extract data for all markets and bets
market_data = []
for market in response:
    overall_data = extract_overall_market_data(market)
    for bet in market.get('markets', []):
        bet_data = extract_bet_data(bet)
        combined_data = {**overall_data, **bet_data}
        market_data.append(combined_data)

# Create DataFrame
df = pd.DataFrame(market_data)

# Sort by Overall Volume in descending order
df = df.sort_values('Overall Volume', ascending=False)

# Display the DataFrame
print(df)

# Display basic statistics
print("\nBasic Statistics:")
print(df.describe())

# Display top markets by volume
print("\nTop 5 Markets by Volume:")
print(df.nlargest(5, 'Bet Volume')[['Title', 'Bet Question', 'Overall Volume', 'Bet Volume', 'Yes Price', 'No Price']])

    Market ID                                        Title  \
17     903193            Presidential Election Winner 2024   
11     903193            Presidential Election Winner 2024   
22     903193            Presidential Election Winner 2024   
21     903193            Presidential Election Winner 2024   
20     903193            Presidential Election Winner 2024   
..        ...                                          ...   
234     11639       Will Biden resign before the election?   
204     11887  Tim Walz replaced as Democratic VP nominee?   
107    903669        Missouri Presidential Election Winner   
106    903669        Missouri Presidential Election Winner   
105    903669        Missouri Presidential Election Winner   

                 End Date  Overall Liquidity  Overall Volume  \
17   2024-11-04T12:00:00Z       5.023445e+08    2.103954e+09   
11   2024-11-04T12:00:00Z       5.023445e+08    2.103954e+09   
22   2024-11-04T12:00:00Z       5.023445e+08    2.103954e+09   

### Polymarket/Betfair arbs

In [121]:
r = requests.get("https://gamma-api.polymarket.com/events?closed=false")
response = r.json()

market_data = []
for market in response:
    overall_data = extract_overall_market_data(market)
    for bet in market.get('markets', []):
        bet_data = extract_bet_data(bet)
        combined_data = {**overall_data, **bet_data}
        market_data.append(combined_data)

# Create DataFrame
polymarket_df = pd.DataFrame(market_data)

In [122]:
# Create a price filter for market data
price_filter = betfairlightweight.filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# List to store DataFrames
dataframes_list_bf = []

# Loop through each market ID and fetch market book data
for market_id in market_ids:
    # Request market book for each market ID
    market_books = client.betting.list_market_book(
        market_ids=[market_id],
        price_projection=price_filter
    )
    
    # Ensure that market books were returned
    if market_books:
        # Process the first market book (only one is requested)
        market_book = market_books[0]
        
        # Get market name using the market_id
        market_name = market_name_mapping[market_id]
        
        # Process runner books and store in DataFrame, including selection names and market names
        runners_df = process_runner_books(market_book.runners, selection_mapping, market_name, market_id)
        
        # Append the DataFrame to the list
        dataframes_list_bf.append(runners_df)

# Optionally, you can concatenate all dataframes into a single dataframe
betfair_df = pd.concat(dataframes_list_bf, ignore_index=True)


/var/folders/ss/kjxldrfs36qgygzmg25sqy100000gn/T/ipykernel_14957/521768696.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  betfair_df = pd.concat(dataframes_list_bf, ignore_index=True)


In [ ]:
# Load the mappings from CSV files
market_mapping_df = pd.read_csv('../data/market_mapping.csv')
selection_mapping_df = pd.read_csv('../data/selection_mapping.csv')

# Convert mapping dataframes to dictionaries
market_mapping = dict(zip(market_mapping_df['polymarket_market_id'], market_mapping_df['betfair_market_id']))
selection_mapping = dict(zip(selection_mapping_df['polymarket_bet_id'], selection_mapping_df['betfair_selection_id']))

# Function to apply mapping
def apply_mapping(value, mapping_dict):
    return mapping_dict.get(value, value)

# Apply mappings to Polymarket data
polymarket_df['Mapped Market ID'] = polymarket_df['Market ID'].apply(lambda x: apply_mapping(x, market_mapping))
polymarket_df['Mapped Selection ID'] = polymarket_df['Bet ID'].apply(lambda x: apply_mapping(x, selection_mapping))

# Rename Betfair columns to match
betfair_df = betfair_df.rename(columns={'Market ID': 'Mapped Market ID', 'Selection ID': 'Mapped Selection ID'})

polymarket_df['Mapped Market ID'] = polymarket_df['Mapped Market ID'].astype(str)
polymarket_df['Mapped Selection ID'] = polymarket_df['Mapped Selection ID'].astype(str)
betfair_df['Mapped Market ID'] = betfair_df['Mapped Market ID'].astype(str)
betfair_df['Mapped Selection ID'] = betfair_df['Mapped Selection ID'].astype(str)

# Merge the dataframes
merged_df = pd.merge(polymarket_df, betfair_df, 
                     on=['Mapped Market ID', 'Mapped Selection ID'], 
                     how='outer', 
                     suffixes=('_poly', '_bet'))

In [130]:
# Display the merged dataframe
print(merged_df[merged_df['Market ID']=='903193'])

    Market ID                              Title              End Date  \
786    903193  Presidential Election Winner 2024  2024-11-04T12:00:00Z   
787    903193  Presidential Election Winner 2024  2024-11-04T12:00:00Z   
788    903193  Presidential Election Winner 2024  2024-11-04T12:00:00Z   
789    903193  Presidential Election Winner 2024  2024-11-04T12:00:00Z   
790    903193  Presidential Election Winner 2024  2024-11-04T12:00:00Z   
791    903193  Presidential Election Winner 2024  2024-11-04T12:00:00Z   
792    903193  Presidential Election Winner 2024  2024-11-04T12:00:00Z   
793    903193  Presidential Election Winner 2024  2024-11-04T12:00:00Z   
794    903193  Presidential Election Winner 2024  2024-11-04T12:00:00Z   
795    903193  Presidential Election Winner 2024  2024-11-04T12:00:00Z   
796    903193  Presidential Election Winner 2024  2024-11-04T12:00:00Z   
797    903193  Presidential Election Winner 2024  2024-11-04T12:00:00Z   
798    903193  Presidential Election W